In [1]:
# List of file names
file_names = [
    'hp_res.csv',
    'hp_input.csv',
    'evcs_res.csv',
    'evcs_input.csv',
    'pv_res.csv',
    'pv_input.csv',
    'em_input.csv',
    'load_res.csv',
    'load_input.csv',
    'storage_res.csv',
    'storage_input.csv'
]


In [2]:
import os
import pandas as pd

# Get the folder path where the files are located
folder_inputs = r'C:\tmp\20231121_flex_testminigrid_2023-11-21_08-37-58\input\grid'

# Filenames
file_names = [
    'em_input.csv',
    'evcs_input.csv',
    'hp_input.csv',
    'pv_input.csv',
    'load_input.csv',
    'storage_input.csv'
]

# Read input data
print('Input data loading')

data_input_em = pd.read_csv(os.path.join(folder_inputs,file_names[0]),index_col='uuid')


# Remove em-system
data_input_em = data_input_em[~data_input_em['id'].str.contains('em-system')]

data_input_ev = pd.read_csv(os.path.join(folder_inputs, file_names[1]),index_col='uuid')
data_input_hp = pd.read_csv(os.path.join(folder_inputs, file_names[2]),index_col='uuid')
data_input_pv = pd.read_csv(os.path.join(folder_inputs, file_names[3]),index_col='uuid')
data_input_lo = pd.read_csv(os.path.join(folder_inputs, file_names[4]),index_col='uuid')
data_input_bs = pd.read_csv(os.path.join(folder_inputs, file_names[5]),index_col='uuid')
data_input_em

Input data loading


,id,node,connected_assets,control_strategy,q_characteristics,operates_from,operates_until,operator
uuid,,,,,,,,
82a5963e-8b34-4909-a6fe-f8a5d630714f,House_15_1,e3c3c6a3-c383-4dbb-9b3f-a14125615386,af6337ae-17f8-498a-934b-1e7a73fd90bc 14214974-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
a20a27bc-d8dc-474a-a067-13768efa12b0,House_15_2,e3c3c6a3-c383-4dbb-9b3f-a14125615386,16ff9d48-d412-4c3a-a3a5-5f1e637adff4 83b88129-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
368b7d20-246c-47b4-99d9-59ecb09d262f,House_15_3,e3c3c6a3-c383-4dbb-9b3f-a14125615386,0cb2a585-82ff-4999-a27a-d056c8f4efd4 d7765490-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
7578de94-39da-461a-a7e9-35887d5cd069,House_15_4,e3c3c6a3-c383-4dbb-9b3f-a14125615386,7e8cceeb-31c5-44f0-ac6f-8aa45d4c121c d7485ea6-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
32bdf883-bf93-41d4-8df0-341c643c0dde,House_15_5,e3c3c6a3-c383-4dbb-9b3f-a14125615386,a30bb0a8-e3c1-4575-8d85-a0f4743e0722 53b6196d-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
fc066082-5114-46dc-81e9-aadd5bdb5a5c,House_15_6,e3c3c6a3-c383-4dbb-9b3f-a14125615386,b73f679c-8972-40b2-ab10-d4fe6de92b6a 61f09e90-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
c91910c7-8052-439d-bbb5-82a9a6f33805,House_15_7,e3c3c6a3-c383-4dbb-9b3f-a14125615386,4dfaf6e1-10be-4e00-9198-8c47a93e9902 b85392b3-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
97cbb95f-ba3d-4cf2-ab8a-ebf6cd581bf2,House_15_8,e3c3c6a3-c383-4dbb-9b3f-a14125615386,7b47ea8e-97fc-4da9-92ab-41fc21608f93 4cd55c5e-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN
da9b5c51-9fcc-4b3f-9fcb-331b5fe9071e,House_15_9,e3c3c6a3-c383-4dbb-9b3f-a14125615386,3c0f8fb3-2a76-4b8f-a28f-807d7826a8a9 da6dffd6-...,self_optimization,"cosPhiFixed:{(0.0, 0.9)}",NaN,NaN,NaN


In [3]:

def create_em_to_systemparticipant_dict(em_list, system_participants_list):
    
    matching_uuids_dict = dict()
    
    for index, row in system_participants_list.iterrows():
        for em_index, em_row in em_list.iterrows():
            connected_assets = em_row['connected_assets']
            
            connected_assets_list = connected_assets.strip('[]').replace("'", "").split(', ') if isinstance(connected_assets, str) else None
                       
            if isinstance(connected_assets_list, list) and index in connected_assets:
                if em_index not in matching_uuids_dict:
                    matching_uuids_dict[em_index] = index
                else:
                    # If there are multiple occurrences, store them as a list
                    if not isinstance(matching_uuids_dict[em_index], list):
                        raise ValueError("uuid is included more then once in em")
    return matching_uuids_dict
    
emToHp = create_em_to_systemparticipant_dict(data_input_em,data_input_hp)    
emToEv = create_em_to_systemparticipant_dict(data_input_em,data_input_ev)
emToBs = create_em_to_systemparticipant_dict(data_input_em,data_input_bs)
emToPv = create_em_to_systemparticipant_dict(data_input_em,data_input_pv)
emToLo = create_em_to_systemparticipant_dict(data_input_em,data_input_lo)




In [4]:
import csv
import datetime

def read_sort_time_series(filename, data_input):
     # Read the CSV file
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        hdr = next(reader)
        num = len(hdr) - 1  # Number of columns
        
        # Interpret timestamp
        timestamp = [datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S') for row in reader]

    # Get UnitIDs
    unitIDs = hdr[1:]
    nUnits = len(unitIDs)

    # Read data and handle exceptions
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        try:
            time_series = [[float(val) for val in row[1:]] for row in reader]
        except ValueError:
            data = list(reader)
            L = [len(row) - 1 for row in data]
            time_series = []
            for i, row in enumerate(data):
                vals = [float(val) if val else float('nan') for val in row[1:]]
                vals.extend([float('nan')] * (max(L) - L[i]))
                time_series.append(vals)

    # Adjust sorting based on input data
    time_series_sort = [[float('nan') for _ in range(nUnits)] for _ in range(len(time_series))]
    for i in range(nUnits):
        for j, uuid in enumerate(inputs['uuid']):
            if uuid in unitIDs:
                xx = unitIDs.index(uuid)
                if xx < nUnits:
                    time_series_sort[:, i] = time_series[:, xx]

    return time_series_sort

In [5]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime
from pypsdm.processing.series import (
    add_series,)
from pypsdm.models.result import (
    entity,)
from pypsdm.models import (
    enums,)


folder_res = r'C:\tmp\20231121_flex_testminigrid_2023-11-21_08-37-58\rawOutputData'
filename_hp = "hp_res.csv"
filename_ev = "evcs_res.csv"
filename_pv = "pv_res.csv"
filename_lo = "load_res.csv"
filename_bs = "storage_res.csv"

filename = os.path.join(folder_res, filename_hp)   
     
timeseries_data = pd.read_csv(os.path.join(folder_res, filename_hp))
#print(timeseries_data)

#print(len(timeseries_data.loc[1,:].values[4]))



# Function to add seconds if they're missing from the timestamp
# Function to add seconds if they're missing from the timestamp
def add_seconds(timestamp):
    if len(timestamp) == 22 and timestamp.endswith('Z[UTC]'):  # Check if seconds are missing and ends with 'Z[UTC]'
        timestamp = timestamp[:-6] + ':00' + timestamp[-6:]  # Insert ':00' before 'Z[UTC]'
    return timestamp


# Add seconds to all timestamps
timeseries_data['time'] = timeseries_data['time'].apply(add_seconds)

# Convert 'time' column to datetime format with optional seconds
timeseries_data['time'] = pd.to_datetime(timeseries_data['time'], format='%Y-%m-%dT%H:%M:%SZ[UTC]')

# Format 'time' column to the desired string format
timeseries_data['time'] = timeseries_data['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

#specific_model = 'd3413988-ebcd-4aff-b5b5-d70364a3b7a2'  # Replace this with the specific entry you want to filter for
#filtered_df = timeseries_data[timeseries_data['input_model'] == specific_model]
#filtered_df.set_index(['time','input_model'],inplace=True)
timeseries_data.set_index(['time','input_model'],inplace=True)



#endtime = datetime(2020,1,1)
endtime = datetime(2019,1,15)

#d34=entity.ResultEntities.build(enums.SystemParticipantsEnum.HEAT_PUMP,specific_model, timeseries_data, endtime)
#print(d34)



In [6]:
from pypsdm.models.input.container.participants import SystemParticipantsContainer
from pypsdm.models.result.container.participants import ParticipantsResultContainer
from pypsdm.models import gwr

gwr = gwr.GridWithResults.from_csv('flex_minigrid', folder_inputs,',',folder_res,',', simulation_end=endtime)


In [7]:
gwr.results.participants.hps

PQResultDict(entity_type=<SystemParticipantsEnum.HEAT_PUMP: 'hp'>, entities={'0670158f-0436-40c0-ba85-9d15ce963171': PQResult(entity_type=<SystemParticipantsEnum.HEAT_PUMP: 'hp'>, input_model='0670158f-0436-40c0-ba85-9d15ce963171', name='HP_12_House_15_22', data=                       p    q  q_dot
time                                
2019-01-01 00:00:00  0.0  0.0    0.0
2019-01-01 00:15:00  0.0  0.0    0.0
2019-01-01 00:30:00  0.0  0.0    0.0
2019-01-01 00:45:00  0.0  0.0    0.0
2019-01-01 01:00:00  0.0  0.0    0.0
...                  ...  ...    ...
2019-01-14 23:00:00  0.0  0.0    0.0
2019-01-14 23:15:00  0.0  0.0    0.0
2019-01-14 23:30:00  0.0  0.0    0.0
2019-01-14 23:45:00  0.0  0.0    0.0
2019-01-15 00:00:00  0.0  0.0    0.0

[1357 rows x 3 columns]), '3f365251-f212-4a02-843c-25b8f0c18cbf': PQResult(entity_type=<SystemParticipantsEnum.HEAT_PUMP: 'hp'>, input_model='3f365251-f212-4a02-843c-25b8f0c18cbf', name='HP_11_House_15_21', data=                       p    q  q_dot
time  

In [8]:
# Vorhandene Fälle
# PV immer an
# 1. Spalte: BS
# 2. Spalte: EV
# 3. Spalte: HP vorhanden
# 4. Spalte: neue HP & Dämmung


cases = [
    (1, 1, 1, 1),
    (1, 1, 1, 0),
    (1, 1, 0, 1),
    (1, 1, 0, 0),
    (1, 0, 1, 1),
    (1, 0, 1, 0),
    (1, 0, 0, 1),
    (1, 0, 0, 0),
    (0, 1, 1, 1),
    (0, 1, 1, 0),
    (0, 1, 0, 1),
    (0, 1, 0, 0),
    (0, 0, 1, 1),
    (0, 0, 1, 0),
    (0, 0, 0, 1),
    (0, 0, 0, 0),
]

# Resultierende Fälle für GLZ => Remove wenn 3. Spalte == 1 und Spalte 1 und 2 nicht 1

filtered_cases = [case for case in cases if case[2] != 0 or case[0] == 1 or case[1] == 1]

filtered_cases

# Fälle Binär-codiert (1,1,1,1 ==15)
# Gemeinsame Analyse von 
# Fall 14 und Fall 15 (identisch nur neue oder alte WP & Dämmung) => BS + EV + WP
# 13 und 12 (identisch) => BS + EV
# 11 und 10 (identisch nur neue oder alte WP & Dämmung) BS + WP 
# 9 und 8 (identisch) => Nur BS
# 7 und 6 (identisch nur neue oder alte WP & Dämmung) => EV + WP 
# 4 und 5 (identisch) => Nur EV
# 2 und 3 (identisch nur neue oder alte WP & Dämmung) => Nur WP




[(1, 1, 1, 1),
 (1, 1, 1, 0),
 (1, 1, 0, 1),
 (1, 1, 0, 0),
 (1, 0, 1, 1),
 (1, 0, 1, 0),
 (1, 0, 0, 1),
 (1, 0, 0, 0),
 (0, 1, 1, 1),
 (0, 1, 1, 0),
 (0, 1, 0, 1),
 (0, 1, 0, 0),
 (0, 0, 1, 1),
 (0, 0, 1, 0)]

In [9]:

folder_glz_cases = r'C:\tmp\20231121_flex_testminigrid_2023-11-21_08-37-58\input\GLZ'
file_name_glz_cases = 'em_to_case_dict.csv'
em_cases_dict = pd.read_csv(os.path.join(folder_glz_cases, file_name_glz_cases), index_col=0).rename_axis(index='em_uuid')

em_cases_dict

,0,1,2,3
em_uuid,,,,
82a5963e-8b34-4909-a6fe-f8a5d630714f,1,1,1,1
a20a27bc-d8dc-474a-a067-13768efa12b0,1,1,1,1
368b7d20-246c-47b4-99d9-59ecb09d262f,1,1,1,1
7578de94-39da-461a-a7e9-35887d5cd069,1,1,1,1
32bdf883-bf93-41d4-8df0-341c643c0dde,1,1,1,1
fc066082-5114-46dc-81e9-aadd5bdb5a5c,1,1,1,0
c91910c7-8052-439d-bbb5-82a9a6f33805,1,1,1,0
97cbb95f-ba3d-4cf2-ab8a-ebf6cd581bf2,1,1,1,0
da9b5c51-9fcc-4b3f-9fcb-331b5fe9071e,1,1,1,0


In [97]:
ems_grid = gwr.grid.participants.ems
ems_results=gwr.results.participants.ems

test_uuid = '82a5963e-8b34-4909-a6fe-f8a5d630714f'

ems_results.get(test_uuid).data





,p,q
time,,
2019-01-01 00:00:00,0.000844,0.000409
2019-01-01 00:15:00,0.000759,0.000368
2019-01-01 00:30:00,0.000686,0.000332
2019-01-01 00:45:00,0.000623,0.000302
2019-01-01 01:00:00,0.000577,0.000279
...,...,...
2019-01-14 23:00:00,0.001237,0.000599
2019-01-14 23:15:00,0.001139,0.000552
2019-01-14 23:30:00,0.001042,0.000505


In [98]:
from pypsdm.processing.series import quarter_hourly_mean_resample

resamp =  quarter_hourly_mean_resample(gwr.results.participants.ems.p)
resamp

,1d5b5cdf-abac-461d-873e-1f7ffa251af4,268ac12a-5617-4de7-a63a-27a2ae13fbf9,2717884a-acd0-475a-809b-e4c3e904aa1e,2a103ae2-9dc4-44f7-b31e-94fd0d8419df,32bdf883-bf93-41d4-8df0-341c643c0dde,368b7d20-246c-47b4-99d9-59ecb09d262f,3cc0c421-1a21-467e-b609-e5b90ac0107d,3f846f16-d139-47d7-ab3f-5cdd0c695d86,4c0f6062-6bf7-4b1a-9af4-0501e4b7a271,53d9ead4-a565-4a00-9e31-4b6336d94503,...,b62da8a1-8b4c-47cb-8bd7-8a9e5bc71ad2,c4e4b6fc-b842-41ae-8a80-4f82a22ec3f9,c91910c7-8052-439d-bbb5-82a9a6f33805,da9b5c51-9fcc-4b3f-9fcb-331b5fe9071e,e1600448-5fa5-4ba2-8938-eaf768f2d7fa,e2a21ad1-c3a8-436d-8473-ea7613c31694,e693fac5-2693-4712-bab8-814f1c06af2b,e791a888-38dd-481a-9558-e15e3780d887,f596c0f1-efc1-4314-a92f-254469585926,fc066082-5114-46dc-81e9-aadd5bdb5a5c
time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,...,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844
2019-01-01 00:15:00,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,...,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759
2019-01-01 00:30:00,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,...,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686
2019-01-01 00:45:00,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,...,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623
2019-01-01 01:00:00,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,...,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-14 23:00:00,0.001237,0.003822,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,...,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237
2019-01-14 23:15:00,0.001139,0.003723,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,...,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139
2019-01-14 23:30:00,0.001042,0.003627,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,...,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.012042


In [99]:
def getEmInstalledCapacatiyFromUuid(em_uuid, ems_grid):

    load_srated=0
    hp_srated=0
    evcs_srated=0
    bs_srated=0
    pv_srated=0
    total_s_rated_em_load_direction = 0
    total_s_rated_em_feedin_direction = 0
    em_installed_capacity = pd.DataFrame(columns=['s_rated_em_load_direction', 's_rated_em_feedin_direction'])   
    
    for conected_asset in ems_grid.connected_assets.get(em_uuid):
        if gwr.grid.participants.loads.__contains__(conected_asset):
            load_srated=gwr.grid.participants.loads.get(conected_asset)['s_rated']     
        if gwr.grid.participants.hps.__contains__(conected_asset):
            hp_srated=gwr.grid.participants.hps.get(conected_asset)['s_rated']
        if gwr.grid.participants.evcs.__contains__(conected_asset):
            evcs_srated=gwr.grid.participants.evcs.get(conected_asset)['power']
        if gwr.grid.participants.storages.__contains__(conected_asset):
            bs_srated=gwr.grid.participants.storages.get(conected_asset)['s_rated']
        if gwr.grid.participants.pvs.__contains__(conected_asset):
            pv_srated=gwr.grid.participants.pvs.get(conected_asset)['s_rated']
            
    ##### TODO FIXME: LOAD INCLUDED HERE OR NOT????????????????????????????????????????????????????????????????????
    s_rated_em_load_direction = load_srated + hp_srated + evcs_srated + bs_srated
    s_rated_em_feedin_direction = evcs_srated + bs_srated + pv_srated
    em_installed_capacity.loc[em_uuid]= [s_rated_em_load_direction,s_rated_em_feedin_direction]
                
    return em_installed_capacity

In [100]:
print(em_cases_dict)

                                      0  1  2  3
em_uuid                                         
82a5963e-8b34-4909-a6fe-f8a5d630714f  1  1  1  1
a20a27bc-d8dc-474a-a067-13768efa12b0  1  1  1  1
368b7d20-246c-47b4-99d9-59ecb09d262f  1  1  1  1
7578de94-39da-461a-a7e9-35887d5cd069  1  1  1  1
32bdf883-bf93-41d4-8df0-341c643c0dde  1  1  1  1
fc066082-5114-46dc-81e9-aadd5bdb5a5c  1  1  1  0
c91910c7-8052-439d-bbb5-82a9a6f33805  1  1  1  0
97cbb95f-ba3d-4cf2-ab8a-ebf6cd581bf2  1  1  1  0
da9b5c51-9fcc-4b3f-9fcb-331b5fe9071e  1  1  1  0
268ac12a-5617-4de7-a63a-27a2ae13fbf9  1  1  1  0
54c41a9b-071b-4ac4-ba52-daa5adf82281  1  1  0  1
e791a888-38dd-481a-9558-e15e3780d887  1  1  0  1
a03331b0-29a1-4547-a439-d6ec01c7b5ec  1  1  0  1
2717884a-acd0-475a-809b-e4c3e904aa1e  1  1  0  1
3f846f16-d139-47d7-ab3f-5cdd0c695d86  1  1  0  1
5468b04e-ade9-4443-86ac-365b443b0dc2  1  1  0  0
b62da8a1-8b4c-47cb-8bd7-8a9e5bc71ad2  1  1  0  0
e1600448-5fa5-4ba2-8938-eaf768f2d7fa  1  1  0  0
53d9ead4-a565-4a00-9

In [101]:
def getCasesFromConditions(dict, cond1, cond2, cond3, cond4):
    filtered_uuids = []
    
    # Loop through the dictionary items and filter based on conditions
    for uuid, values in dict.iterrows():
        if values[0] == cond1 and values[1] == cond2 and values[2] == cond3 and values[3] == cond4:
            filtered_uuids.append(uuid)
    return  filtered_uuids


In [105]:
test = getCasesFromConditions(em_cases_dict, 1,1,1,1)
print(test)

['82a5963e-8b34-4909-a6fe-f8a5d630714f', 'a20a27bc-d8dc-474a-a067-13768efa12b0', '368b7d20-246c-47b4-99d9-59ecb09d262f', '7578de94-39da-461a-a7e9-35887d5cd069', '32bdf883-bf93-41d4-8df0-341c643c0dde']


In [106]:

for item in test:
    getEmInstalledCapacatiyFromUuid(item, ems_grid)
    

In [111]:
em_installed_capacity_res = pd.DataFrame(columns=['s_rated_em_load_direction', 's_rated_em_feedin_direction'])   
for item in test:
    val = getEmInstalledCapacatiyFromUuid(item, ems_grid)
    em_installed_capacity_res.loc[item]= [val.values[0][0], val.values[0][1]]
    

em_installed_capacity_res

,s_rated_em_load_direction,s_rated_em_feedin_direction
82a5963e-8b34-4909-a6fe-f8a5d630714f,39.234324,41.823098
a20a27bc-d8dc-474a-a067-13768efa12b0,37.890073,39.287530
368b7d20-246c-47b4-99d9-59ecb09d262f,38.541396,39.825692
7578de94-39da-461a-a7e9-35887d5cd069,38.173961,37.580415
32bdf883-bf93-41d4-8df0-341c643c0dde,38.356602,38.752785


In [86]:
print(em_installed_capacity_res.loc['a20a27bc-d8dc-474a-a067-13768efa12b0','s_rated_em_load_direction'])


37.89007324623006


In [18]:
uu = '019f1307-e73e-40b2-9277-22f960aafc1e'

gwr.results.participants.evcs.get(uu).p.max()

0.011

In [114]:
df_res = quarter_hourly_mean_resample(gwr.results.participants.ems.p)
df_res = df_res.filter(em_installed_capacity_res.index)

In [115]:
for column in df_res.columns:
    print(column)

82a5963e-8b34-4909-a6fe-f8a5d630714f
a20a27bc-d8dc-474a-a067-13768efa12b0
368b7d20-246c-47b4-99d9-59ecb09d262f
7578de94-39da-461a-a7e9-35887d5cd069
32bdf883-bf93-41d4-8df0-341c643c0dde


In [116]:
def normalize_dataframe_min_max(df, min_val, max_val):
    normalized_df = df  # Create a copy of the original DataFrame
    
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numerical data (you may want to refine this condition based on your specific DataFrame)
        if df[column].dtype in [int, float]:
            #    min_val = df[column].min()  # Calculate the minimum value of the column
            #    max_val = df[column].max()  # Calculate the maximum value of the column
            
            # Min-Max normalization formula: (x - min) / (max - min)
            normalized_df[column] = (df[column] - min_val) / (max_val - min_val)
    
    return normalized_df

In [128]:
from pypsdm.processing.series import quarter_hourly_mean_resample

for item in gwr.results.participants.evcs:
    test=(quarter_hourly_mean_resample(gwr.results.participants.evcs.get(item).p))
    

In [218]:
em_installed_capacity_res= em_installed_capacity_res.rename_axis('uuid')
em_installed_capacity_res

,s_rated_em_load_direction,s_rated_em_feedin_direction
uuid,,
82a5963e-8b34-4909-a6fe-f8a5d630714f,39.234324,41.823098
a20a27bc-d8dc-474a-a067-13768efa12b0,37.890073,39.287530
368b7d20-246c-47b4-99d9-59ecb09d262f,38.541396,39.825692
7578de94-39da-461a-a7e9-35887d5cd069,38.173961,37.580415
32bdf883-bf93-41d4-8df0-341c643c0dde,38.356602,38.752785


In [326]:
uuid_to_search = '32bdf883-bf93-41d4-8df0-341c643c0dde'  # Replace with the UUID you're searching for

# Filter the DataFrame based on the UUID
filtered_row = em_installed_capacity_res[em_installed_capacity_res.index == uuid_to_search]

if not filtered_row.empty:
    first_column_value = filtered_row.iloc[0, 0]  # Assuming the first column is at index 0
    print("Value from the first column:", first_column_value)
else:
    print("UUID not found in the DataFrame")


Value from the first column: 38.35660233400342


In [327]:
import os

import numpy as np
import pandas
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from pypsdm.processing.series import quarter_hourly_mean_resample


"""
 Gleichzeitigkeit
"""

def calculate_gz_curve(df,df_inst, len_curve, num_mc):
    sim_curve = pd.DataFrame(np.zeros((len_curve, 1)))
    quantile_95 = pd.DataFrame(np.zeros((len_curve, 1)))
    
    
    division_factors = em_installed_capacity_res

    for n in range(1, len_curve + 1):
        temp_sim = np.zeros(num_mc)
        print("Calculate GZ for ev-number "+str(n))
        for mc in range(num_mc):
            profile_row = np.random.choice(df.index, size=n, replace=False)
            print('profile_row: ' + profile_row.__str__())
            mc_profile = df.loc[profile_row, :]
            print('mc_profile: ' + mc_profile.__str__())
            temp_sim[mc] = (mc_profile.sum(axis=0).max() / n)
            print('temp_sim_entry')
            print(mc_profile.sum(axis=0).max() / n)
            print('temp_sim')
            print(temp_sim)
            print('temp_sim_max')
            print(temp_sim.max())
        sim_curve.iloc[n - 1, 0] = temp_sim.max()
        quantile_95.iloc[n - 1, 0] = np.percentile(temp_sim, 95, method='linear')



    return sim_curve, quantile_95


In [328]:
"""
Kurvenanpassung
"""

# https://www.askpython.com/python/examples/curve-fitting-in-python
# Die Kurvenanpassung wird nur für den Bereich vorgenommen, an dem die Funktion differenzierbar ist (wenn GZ nicht mehr 1)

# allgemeine Gleichung der erweiterten GZ-Funktion

def fit_function(n, *args):
    a, b, c = args
    return a * np.power(n, -b) + c


def fit_sim_curve(x_values, y_values):
    # Initialisierung mit Werten EV ungesteuert Dis Kippelt
    p0 = [1.785, 0.631, 0.112]
    popt, pcov = curve_fit(fit_function, x_values, y_values, p0, maxfev=10000)
    # Anzahl der Datenpunkte
    n = len(y_values)
    # Anzahl der Fit-Parameter
    p = len(popt)
    # Mittelwert der abhängigen Variable
    y_mean = np.mean(y_values)
    # Regressionsabweichungen berechnen
    residuals = y_values - fit_function(x_values, *popt)
    # Summe der quadrierten Regressionsabweichungen
    ss_res = np.sum(residuals**2)
    # Gesamtsumme der quadrierten Abweichungen
    ss_tot = np.sum((y_values - y_mean)**2)
    # Bestimmtheitsmaß berechnen
    r_squared = 1 - (ss_res / (n - p - 1)) / (ss_tot / (n - 1))
    return popt, pcov, r_squared

# popt ist ein Array aus den optimalen Parametern, sodass die Summe der Fehlerquadrate minimiert ist
# pcov ist die zugehörige Kovarianzmatrix

In [330]:
"""
 Initialisierung
"""
# Parameter, die einzustellen sind:
num_vehicles = 100
#len_curve = 150  # Ziel: 150, gibt die maximale Anzahl an EV innerhalb der GZ-Kurve an
len_curve = 3  # Ziel: 150, gibt die maximale Anzahl an EV innerhalb der GZ-Kurve an
num_mc = 2  # Ziel: 1000, Anzahl an Monte-Carlo-Iterationen pro Punkt in der GZ-Kurve

# Initialisierung
temp_profile = pd.DataFrame(np.zeros(shape=(0, num_vehicles)))
temp_sim = pd.DataFrame(np.zeros(shape=(num_mc, 1)))

# Einlesen der Daten:


df_resample = quarter_hourly_mean_resample(gwr.results.participants.ems.p)
df_resample = df_resample.filter(em_installed_capacity_res.index)
df_norm = normalize_dataframe_by_installed_capacity(df_resample, em_installed_capacity_res)


# Start: GZ-Kurve
sim_curve, quantile_95_tot = calculate_gz_curve(df_norm, em_installed_capacity_res, len_curve, num_mc)
quantile_95_cut = quantile_95_tot.iloc[:, 0] < 1
quantile_95 = quantile_95_tot.loc[quantile_95_cut, 0].to_numpy()
quantile_95_indices = pd.Series(range(1, len(quantile_95)+1)).to_numpy()
# Kurvenregression
param_opt, param_cov, r_squared = fit_sim_curve(quantile_95_indices, quantile_95)
a_opt, b_opt, c_opt = param_opt
fitted_sim = fit_function(quantile_95_indices, a_opt, b_opt, c_opt)

x = np.arange(1, len(quantile_95_tot[0]) + 1)
y = np.ones_like(quantile_95_tot[0])
y[quantile_95_tot[0] < 1] = fitted_sim[:len(fitted_sim)]


if 0 == 1:
    """
    Plots
    """
    # Plot 2
    #plt.plot(x, gz_bi, label='V2G (Laden)', color=(0.517, 0.721, 0.094))
    #plt.plot(x, gz_bi_dis, label='V2G (Entladen)', color=(0.3, 0.3, 0.3))
    plt.plot(x, y, label='Unidirektional', color=(0.717, 0.921, 0.294))
    #plt.plot(x_unc, gz_unc, label='Ungesteuertes Laden', color=(0.6, 0.6, 0.6))
    plt.grid(color='lightgrey', linestyle='-')
    plt.xlabel('Anzahl der Fahrzeuge')
    plt.ylim(0, 1.3)
    plt.xlim(0, 155)
    plt.ylabel('Gleichzeitigkeitsfaktor')
    #plt.legend()
    plt.show()
    
    plt.plot(x, y, label='Regressionskurve', color=(0.6, 0.6, 0.6))
    plt.plot((quantile_95_tot.index + 1), quantile_95_tot, '.', label='95% Quantil', color=(0.517, 0.721, 0.094))
    plt.grid(color='lightgrey', linestyle='-')
    plt.xlabel('Anzahl der Fahrzeuge')
    plt.ylim(0, 1.1)
    plt.xlim(0, 155)
    plt.ylabel('Gleichzeitigkeitsfaktor')
    plt.legend()
    plt.show()
    
    # Plot 1
    plt.plot((sim_curve.index + 1), sim_curve, label='Maximale Gleichzeitigkeit', color=(0.6, 0.6, 0.6))
    plt.grid(color='lightgrey', linestyle='-')
    plt.plot((quantile_95_tot.index + 1), quantile_95_tot, '.', label='95% Quantil', color=(0.517, 0.721, 0.094))
    plt.plot(x, y, label='Regressionskurve', color=(0.3, 0.3, 0.3))
    plt.xlabel('Anzahl der Fahrzeuge')
    plt.ylim(0, 1.1)
    plt.xlim(0, 155)
    plt.ylabel('Gleichzeitigkeitsfaktor')
    plt.legend()
    plt.show()
    
    # Plot 3
    plt.plot(x, y, label='95% Quantil', color=(0.517, 0.721, 0.094))
    plt.grid(color='lightgrey', linestyle='-')
    plt.xlabel('Anzahl der Fahrzeuge')
    plt.ylim(0, 1.1)
    plt.xlim(0, 155)
    plt.ylabel('Gleichzeitigkeitsfaktor')
    plt.show()


Calculate GZ for ev-number 1
profile_row: ['2019-01-14T06:00:00.000000000']
mc_profile:                      82a5963e-8b34-4909-a6fe-f8a5d630714f  \
time                                                        
2019-01-14 06:00:00                               0.02188   

                     a20a27bc-d8dc-474a-a067-13768efa12b0  \
time                                                        
2019-01-14 06:00:00                              0.022656   

                     368b7d20-246c-47b4-99d9-59ecb09d262f  \
time                                                        
2019-01-14 06:00:00                              0.022273   

                     7578de94-39da-461a-a7e9-35887d5cd069  \
time                                                        
2019-01-14 06:00:00                              0.022488   

                     32bdf883-bf93-41d4-8df0-341c643c0dde  
time                                                       
2019-01-14 06:00:00                              0.16731

C:\Users\smdafeis\AppData\Local\pypoetry\Cache\virtualenvs\pypsdm-qt54NClq-py3.11\Lib\site-packages\scipy\optimize\_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [284]:
gwr.results.participants.ems.get('32bdf883-bf93-41d4-8df0-341c643c0dde').p

time
2019-01-01 00:00:00    0.000844
2019-01-01 00:15:00    0.000759
2019-01-01 00:30:00    0.000686
2019-01-01 00:45:00    0.000623
2019-01-01 01:00:00    0.000577
                         ...   
2019-01-14 23:00:00    0.001237
2019-01-14 23:15:00    0.001139
2019-01-14 23:30:00    0.001042
2019-01-14 23:45:00    0.000946
2019-01-15 00:00:00    0.000946
Name: p, Length: 1369, dtype: float64

In [292]:
gwr.grid.participants.ems.connected_assets.get('32bdf883-bf93-41d4-8df0-341c643c0dde')

['a30bb0a8-e3c1-4575-8d85-a0f4743e0722',
 '53b6196d-11e7-45d5-b5e3-7afff2aeb44a',
 '438ca8f7-f8f2-4f5e-8214-a2ac8122f0a0',
 '5abd3b40-3553-4a26-90a5-ed234380ed51',
 'e0fd50f1-bbbe-4817-b378-38835d440c0c']

In [322]:
testdf=gwr.results.participants.loads.get('a30bb0a8-e3c1-4575-8d85-a0f4743e0722').p
profile_row = datetime(2019,1,9,0,30)
load=testdf[profile_row]
print('load ' + load.__str__())

testdf=gwr.results.participants.pvs.get('53b6196d-11e7-45d5-b5e3-7afff2aeb44a')
pvs=testdf[profile_row].p.values

print('pv ' + pvs.__str__())


testdf=gwr.results.participants.storages.get('438ca8f7-f8f2-4f5e-8214-a2ac8122f0a0')
print('storage ' + testdf[profile_row].p.values.__str__())

testdf=gwr.results.participants.evcs.get('5abd3b40-3553-4a26-90a5-ed234380ed51')
print('evcs ' + testdf[profile_row].p.values.__str__())

testdf=gwr.results.participants.hps.get('e0fd50f1-bbbe-4817-b378-38835d440c0c')
hps = testdf[profile_row].p.values
print('hps ' + hps.__str__())

summe = load + hps
print(summe)

load 0.0006915860014892033
pv [0.]
storage [0.]
evcs [0.]
hps [0.00555915]
[0.00625074]


In [291]:
gwr.results.participants.loads.get()

,0cb2a585-82ff-4999-a27a-d056c8f4efd4,104b01a4-087b-475e-ad64-6f993a4bc861,11239ca9-ea82-4ecd-8258-711e7b219ba6,16ff9d48-d412-4c3a-a3a5-5f1e637adff4,1e319e79-8468-49ca-ba01-f28931f75313,3685a343-c547-412c-a844-26cd574540c8,39a820fe-82d7-4fdd-a78d-5daf9a5b0250,3c0f8fb3-2a76-4b8f-a28f-807d7826a8a9,402ed403-617b-459b-82f7-530941973231,479d59f3-2329-48e7-94b9-e9d6d65687b1,...,b73f679c-8972-40b2-ab10-d4fe6de92b6a,b9c2574d-782e-4d46-b079-9e3e3a7e0c66,c0bb7689-88bd-4ca4-a81f-16f4ea2a221b,cdf23a76-9091-4968-93f1-444cc8c4e7bd,d7947073-af0a-4f55-bf5c-171178138b79,df2f6c58-8bb2-40b5-a966-b5714ed609b7,df34ab8a-8927-4e2d-a741-bc8d274e3841,e4547641-03e7-437d-980e-fee1e9152057,f306ba1d-c636-4f08-99ce-0cbc11fb746a,fdcbdd9a-cf76-4ec9-b5d7-a68d4b9acdb8
time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,...,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844,0.000844
2019-01-01 00:15:00,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,...,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759,0.000759
2019-01-01 00:30:00,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,...,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686,0.000686
2019-01-01 00:45:00,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,...,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623,0.000623
2019-01-01 01:00:00,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,...,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577,0.000577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-14 23:00:00,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,...,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237,0.001237
2019-01-14 23:15:00,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,...,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139,0.001139
2019-01-14 23:30:00,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,...,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042,0.001042


## Code von Vicky


In [323]:
import os

import numpy as np
import pandas
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from pypsdm.processing.series import quarter_hourly_mean_resample



matplotlib.use('TkAgg')
from scipy.optimize import curve_fit

"""
 Gleichzeitigkeit
"""

def calculate_gz_curve(df_norm, len_curve, num_mc):
    sim_curve = pd.DataFrame(np.zeros((len_curve, 1)))
    quantile_95 = pd.DataFrame(np.zeros((len_curve, 1)))

    for n in range(1, len_curve + 1):
        temp_sim = np.zeros(num_mc)
        print("Calculate GZ for ev-number "+str(n))
        for mc in range(num_mc):
            profile_row = np.random.choice(df_norm.index, size=n, replace=False)
            mc_profile = df_norm.loc[profile_row, :]
            temp_sim[mc] = mc_profile.sum(axis=0).max() / n
        sim_curve.iloc[n - 1, 0] = temp_sim.max()
        quantile_95.iloc[n - 1, 0] = np.percentile(temp_sim, 95, method='linear')



    return sim_curve, quantile_95


"""
Kurvenanpassung
"""

# https://www.askpython.com/python/examples/curve-fitting-in-python
# Die Kurvenanpassung wird nur für den Bereich vorgenommen, an dem die Funktion differenzierbar ist (wenn GZ nicht mehr 1)

# allgemeine Gleichung der erweiterten GZ-Funktion

def fit_function(n, *args):
    a, b, c = args
    return a * np.power(n, -b) + c


def fit_sim_curve(x_values, y_values):
    # Initialisierung mit Werten EV ungesteuert Dis Kippelt
    p0 = [1.785, 0.631, 0.112]
    popt, pcov = curve_fit(fit_function, x_values, y_values, p0, maxfev=10000)
    # Anzahl der Datenpunkte
    n = len(y_values)
    # Anzahl der Fit-Parameter
    p = len(popt)
    # Mittelwert der abhängigen Variable
    y_mean = np.mean(y_values)
    # Regressionsabweichungen berechnen
    residuals = y_values - fit_function(x_values, *popt)
    # Summe der quadrierten Regressionsabweichungen
    ss_res = np.sum(residuals**2)
    # Gesamtsumme der quadrierten Abweichungen
    ss_tot = np.sum((y_values - y_mean)**2)
    # Bestimmtheitsmaß berechnen
    r_squared = 1 - (ss_res / (n - p - 1)) / (ss_tot / (n - 1))
    return popt, pcov, r_squared

# popt ist ein Array aus den optimalen Parametern, sodass die Summe der Fehlerquadrate minimiert ist
# pcov ist die zugehörige Kovarianzmatrix

"""
 Initialisierung
"""
# Parameter, die einzustellen sind:
num_vehicles = 1000
#len_curve = 150  # Ziel: 150, gibt die maximale Anzahl an EV innerhalb der GZ-Kurve an
len_curve = 4  # Ziel: 150, gibt die maximale Anzahl an EV innerhalb der GZ-Kurve an
num_mc = 1000  # Ziel: 1000, Anzahl an Monte-Carlo-Iterationen pro Punkt in der GZ-Kurve

# Initialisierung
temp_profile = pd.DataFrame(np.zeros(shape=(0, num_vehicles)))
temp_sim = pd.DataFrame(np.zeros(shape=(num_mc, 1)))

# Einlesen der Daten:

# Hier müssen die optimierten Profile gemäß der Durchmischung von Ladestrategien zusammengestellt werden


if 0 == 1 :
    share_unc_65 = np.load('MasterarbeitMain/opt_uni_charge_total.npy')
    share_unc_65 = share_unc_65[0:3250, :]
    share_unc_65 = pd.DataFrame(share_unc_65)
    share_bi = np.load('MasterarbeitMain/opt_bi_charge_total.npy')
    share_bi_35 = share_bi[0:1750, :]
    share_bi_35 = pd.DataFrame(share_bi_35)
    share_unc_65 = share_unc_65

    df_norm = (pd.concat([share_unc_65, share_bi_35])).reset_index(drop=True)

df_evcs = gwr.results.participants.evcs.p

#df_norm = normalize_dataframe(df_evcs, 0.0, 0.011)

df_resample = quarter_hourly_mean_resample(df_evcs)
df_norm = normalize_dataframe(df_resample, 0,0.011)

#df_norm = quarter_hourly_mean_resample(normalize_dataframe(gwr.results.participants.evcs.p,0, 0.011))


# Start: GZ-Kurve
sim_curve, quantile_95_tot = calculate_gz_curve(df_norm, len_curve, num_mc)
quantile_95_cut = quantile_95_tot.iloc[:, 0] < 1
quantile_95 = quantile_95_tot.loc[quantile_95_cut, 0].to_numpy()
quantile_95_indices = pd.Series(range(1, len(quantile_95)+1)).to_numpy()
# Kurvenregression
param_opt, param_cov, r_squared = fit_sim_curve(quantile_95_indices, quantile_95)
a_opt, b_opt, c_opt = param_opt
fitted_sim = fit_function(quantile_95_indices, a_opt, b_opt, c_opt)

x = np.arange(1, len(quantile_95_tot[0]) + 1)
y = np.ones_like(quantile_95_tot[0])
y[quantile_95_tot[0] < 1] = fitted_sim[:len(fitted_sim)]



"""
Plots
"""
# Plot 2
#plt.plot(x, gz_bi, label='V2G (Laden)', color=(0.517, 0.721, 0.094))
#plt.plot(x, gz_bi_dis, label='V2G (Entladen)', color=(0.3, 0.3, 0.3))
plt.plot(x, y, label='Unidirektional', color=(0.717, 0.921, 0.294))
#plt.plot(x_unc, gz_unc, label='Ungesteuertes Laden', color=(0.6, 0.6, 0.6))
plt.grid(color='lightgrey', linestyle='-')
plt.xlabel('Anzahl der Fahrzeuge')
plt.ylim(0, 1.3)
plt.xlim(0, 155)
plt.ylabel('Gleichzeitigkeitsfaktor')
#plt.legend()
plt.show()

plt.plot(x, y, label='Regressionskurve', color=(0.6, 0.6, 0.6))
plt.plot((quantile_95_tot.index + 1), quantile_95_tot, '.', label='95% Quantil', color=(0.517, 0.721, 0.094))
plt.grid(color='lightgrey', linestyle='-')
plt.xlabel('Anzahl der Fahrzeuge')
plt.ylim(0, 1.1)
plt.xlim(0, 155)
plt.ylabel('Gleichzeitigkeitsfaktor')
plt.legend()
plt.show()

# Plot 1
plt.plot((sim_curve.index + 1), sim_curve, label='Maximale Gleichzeitigkeit', color=(0.6, 0.6, 0.6))
plt.grid(color='lightgrey', linestyle='-')
plt.plot((quantile_95_tot.index + 1), quantile_95_tot, '.', label='95% Quantil', color=(0.517, 0.721, 0.094))
plt.plot(x, y, label='Regressionskurve', color=(0.3, 0.3, 0.3))
plt.xlabel('Anzahl der Fahrzeuge')
plt.ylim(0, 1.1)
plt.xlim(0, 155)
plt.ylabel('Gleichzeitigkeitsfaktor')
plt.legend()
plt.show()

# Plot 3
plt.plot(x, y, label='95% Quantil', color=(0.517, 0.721, 0.094))
plt.grid(color='lightgrey', linestyle='-')
plt.xlabel('Anzahl der Fahrzeuge')
plt.ylim(0, 1.1)
plt.xlim(0, 155)
plt.ylabel('Gleichzeitigkeitsfaktor')
plt.show()


Calculate GZ for ev-number 1
Calculate GZ for ev-number 2
Calculate GZ for ev-number 3


KeyboardInterrupt: 

In [22]:
df_evcs

,019f1307-e73e-40b2-9277-22f960aafc1e,0c5e3c7d-772f-42c4-9d5d-ab52311d717d,257ac38a-50e0-4962-9665-9d5202fa35b0,25d22721-2f32-404e-817a-873f6c3d770a,2acff78c-3018-4bb6-b52b-5d0061092100,2ca09423-7c8b-4539-946d-7def95c077ea,3aa8e477-321f-4870-8276-12e8dd468701,3ac0f440-6ae1-41fc-8227-4ee18ab1327d,4542c40f-7678-40fb-97fa-c79849dff4cb,55f9d36d-d923-4c35-8517-9d72fac9f2b3,...,997ebb67-2dde-473d-b210-cc9091812f66,9a6c0edb-ef62-46e4-8cd5-ff7e4ad177c1,9b52ac95-af90-437f-959f-f140c4b8ed97,ace5617a-5aab-4667-94e2-3b87ce1b06af,c3a3da0d-e0a3-48e7-a266-2fa66057f00d,c9d6a77b-4065-4f3c-a00f-25799959afdb,d2686fdc-29e5-421c-bf18-068be890dcee,d7d1e69d-a1c7-48de-868a-0c94d96f9120,e17a4c08-06ba-470a-b2ec-ec88d4ee2f3a,fad29ae8-5f90-49ea-aba7-f9c1e8377d0f
time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-14 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
2019-01-14 23:21:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0
2019-01-14 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.011,0.0,0.0,0.0,0.0,0.0
